In [1]:
1+1


2

In [ ]:
import os
import logging
from dotenv import load_dotenv
from typing import TypedDict, List
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain.agents import Tool
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, END


# 환경 변수
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# logging.basicConfig(level=logging.DEBUG)
logging.getLogger("langchain").setLevel(logging.DEBUG)

# 도구 함수 정의
def add_two(x: str) -> str:
    try:
        return str(float(x) + 2)
    except Exception:
        return "도구 실행 실패: 숫자만 입력해 주세요."


def population_lookup(city: str) -> str:
    data = {
        "서울": "9,515,000명 (2023년 기준)", 
        "부산": "3,343,000명 (2023년 기준)"
        }
    # return data.get(city.strip().lower(), f"{city}의 인구 정보를 찾을 수 없습니다.")
    raise ValueError(f"{city}의 인구 정보를 찾을 수 없습니다.")


# 메시지 상태 정의
class AgentState(TypedDict):
    messages: List[BaseMessage]


# 도구 리스트 정의
tools = [
    Tool(
        name="AddTwo", 
        func=add_two, 
        description="숫자 입력 시 2를 더합니다."
        ),
    Tool(
        name="PopulationLookup",
        func=population_lookup,
        description="도시 인구를 조회합니다.",
        ),
]


# LLM 정의 (ReAct agent는 verbose=True만 있어도 추론 흐름 출력)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, verbose=True)


# ReAct Agent 생성
react_agent = create_react_agent(model=llm, tools=tools, debug=True)


# LangGraph 구성 (단순한 선형 흐름)

# AgentState 타입을 사용하여 상태 그래프 생성
# AgentState는 그래프 전체에서 공유되는 데이터 구조를 정의
graph = StateGraph(AgentState)

# "agent"라는 이름의 노드를 추가하고, react_agent 함수를 실행 로직으로 지정
# 이 노드는 실제 AI 에이전트가 동작하는 핵심 부분
graph.add_node("agent", react_agent)

# 그래프의 시작점을 "agent" 노드로 설정
# 그래프가 실행될 때 가장 먼저 실행되는 노드를 지정
graph.set_entry_point("agent")

# "agent" 노드 실행 완료 후 END로 이동하는 연결선 추가
# 즉, 에이전트가 작업을 완료하면 그래프 실행이 종료됨
graph.add_edge("agent", END)


# 그래프 컴파일

# 정의된 그래프를 실행 가능한 형태로 컴파일
# 이제 app.invoke()로 그래프를 실행할 수 있음
app = graph.compile()


# 테스트 실행
# response = app.invoke({"messages": [HumanMessage(content="seoul")]})

# 최대 반복 횟수 설정
max_iterations = 3
recursion_limit = 2 * max_iterations + 1

# Agent 실행 시 recursion_limit 적용
try:
    response = app.invoke(
        {"messages": [HumanMessage(content="seoul")]},
        {"recursion_limit": recursion_limit},
    )
except Exception:
    print("Agent가 최대 반복 횟수에 도달했습니다.")


# 결과 출력
for m in response["messages"]:
    print(m.__class__.__name__, ":", getattr(m, "content", repr(m)))

[values] {'messages': [HumanMessage(content='seoul', additional_kwargs={}, response_metadata={}, id='52275d74-cf2a-4f5d-bcbc-4d1531b51597')]}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[updates] {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ibFbLeNNBguoXt1Tx68mpfxF', 'function': {'arguments': '{"__arg1":"seoul"}', 'name': 'PopulationLookup'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 84, 'total_tokens': 102, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BpXCFN6L6YOED93LYg76fSPOb7Pp3', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--68cd1f32-462a-44bf-85dc-50c63029a2c1-0', tool_calls=[{'name': 'PopulationLookup', 'args': {'__arg1': 'seoul'}, 'id': 'call_ibFbLeNNBguoXt1Tx68mpfxF', 'type': 'tool_call'}], usage_metadata={'input_tokens': 84, 'output_tokens': 18, 't

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[updates] {'agent': {'messages': [AIMessage(content='서울의 인구 정보를 찾을 수 없습니다. 다른 도시나 정보를 요청해 주시면 도와드리겠습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 121, 'total_tokens': 146, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BpXCG4qIwhBSA4FNrjVydGMWinPsJ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--64133373-4401-4caf-8a68-b5e931d70375-0', usage_metadata={'input_tokens': 121, 'output_tokens': 25, 'total_tokens': 146, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
[values] {'messages': [HumanMessage(content='seoul', additional_kwargs={}, response_metadata={}, id='52275d74